In [1]:
def create_user(conn, login, name):
    llogin = login.lower()
    lock = acquire_lock_with_timeout(conn, f'user:{llogin}', 1)
    
    if not lock:
        return None
    
    if conn.hget('users:', llogin):
        return None

    id = conn.incr('user:id:')
    pipeline = conn.pipeline(True)
    pipeline.hset('user:', llogin, id)
    pipeline.hmset(f'user:{id}', {
        'login': login,
        'id': id,
        'name': name,
        'followers': 0,
        'following': 0,
        'posts': 0,
        'signup': time.time()
    })
    pipeline.execute()
    release_lock(conn, f'user:{llogin}', lock)
    return id

In [2]:
def create_status(conn, uid, message, **data):
    pipeline = conn.pipeline(True)
    
    # Get the user login name.
    pipeline.hget(f'user:{uid}', 'login')
    pipeline.incr('status:id:')
    login, id = pipeline.execute()
    
    # Verify that we have a proper user account before posting.
    if not login:
        return None
    
    data.update({
        'message': message,
        'posted': time.time(),
        'id': id,
        'uid': uid,
        'login': login
    })
    pipeline.hmset(f'status:{id}', data)
    
    # Record the fact that a status message has been posted.
    pipeline.hincrby(f'user:{uid}', 'posts')
    pipeline.execute()
    return id

In [3]:
def get_status_messages(conn, uid, timeline='home:', page=1, count=30):
    statuses = conn.zrevrange(f'{timeline}:{uid}', (page - 1) * count, page * count - 1)
    pipeline = conn.pipeline(True)
    for id in statuses:
        pipeline.hgetall(f'status:{id}')
    
    # Filter will remove any missing status messages that had been previously deleted.
    return filter(None, pipeline.execute())

In [7]:
HOME_TIMELINE_SIZE = 1000
def follow_user(conn, uid, other_uid):
    fkey1 = f'following:{uid}'
    fkey2 = f'followers:{other_uid}'
    
    if conn.zscore(fkey1, other_uid):
        return None
    
    now = time.time()
    
    pipeline = conn.pipeline(True)
    pipeline.zadd(fkey1, other_uid, now)
    pipeline.zadd(fkey2, uid, now)
    
    # Find the size of following and followers.
    pipeline.zcard(fkey1)
    pipeline.zcard(fkey2)
    
    pipeline.zrevrange(f'profile:{other_uid}', 0, HOME_TIMELINE_SIZE-1, withscores=True)
    following, followers, status_and_score = pipeline.execute()[-3:]
    
    pipeline.hset(f'user:{uid}', 'following', following)
    pipeline.hset(f'user:{other_uid}', 'followers', followers)
    if status_and_score:
        pipeline.zadd(f'home:{uid}', **dict(status_and_score))
    pipeline.zremrangebyrank(f'home:{uid}', 0. -HOME_TIMELINE_SIZE-1)
    pipeline.execute()
    return True

In [8]:
def unfollow_user(conn, uid, other_uid):
    fkey1 = f'following:{uid}'
    fkey2 = f'followers:{other_uid}'
    
    if not conn.zscore(fkey1, other_uid):
        return None
    
    pipeline = conn.pipeline(True)
    pipeline.zrem(fkey1, other_uid)
    pipeline.zrem(fkey2, uid)
    pipeline.zcard(fkey1)
    pipeline.zcard(fkey2)
    pipeline.zrevrange(f'profile:{other_uid}', 0, HOME_TIMELINE_SIZE-1)
    following, followers, statuses = pipeline.execute()[-3:]
    
    pipeline.hset(f'user:{uid}', 'following', following)
    pipeline.hset(f'user:{other_uid}', 'followers', followers)
    if statuses:
        pipeline.zrem(f'home:{uid}', *statuses)
    pipeline.execute()
    return True

In [9]:
def post_status(conn, uid, message, **data):
    id = create_status(conn, uid, message, **data)
    if not id:
        return None
    
    posted = conn.hget(f'status:{id}', 'posted')
    if not posted:
        return None
    
    post = {str(id): float(posted)}
    conn.zadd(f'profile:{uid}', **post)
    
    syndicate_status(conn, uid, post)
    return id

In [10]:
POSTS_PER_PASS = 1000
def syndicate_status(conn, uid, post, start=0):
    followers = conn.zrangebyscore(f'followers:{uid}', start, 'inf', start=0, num=POSTS_PER_PASS, withscores=True)
    pipeline = conn.pipeline(False)
    for follower, start in followers:
        pipeline.zadd(f'home:{follower}', **post)
        pipeline.zremrangebyrank(f'home:{follower}', 0, -HOME_TIMELINE_SIZE-1)
    pipeline.execute()
    
    if len(followers) >= POSTS_PER_PASS:
        execute_later(conn, 'default', 'syndicate_status', [conn, uid, post, start])

In [11]:
# A function to delete a previously posted status message.
def delete_status(conn, uid, status_id):
    key = f'status:{status_id}'
    lock = acquire_lock_with_timeout(conn, key, 1)
    if not lock:
        return None
    
    if conn.hget(key, 'uid') != str(uid):
        return None
    
    pipeline = conn.pipeline(True)
    pipeline.delete(key)
    pipeline.zrem(f'profile:{uid}', status_id)
    pipeline.zrem(f'home{uid}', status_id)
    pipeline.hincrby(f'user:{uid}', 'posts', -1)
    pipeline.execute()
    
    release_lock(conn, key, lock)
    return True